In [37]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [77]:
df=pd.read_csv('Churn_Modelling.csv')

In [78]:
df.drop(columns=['RowNumber','CustomerId','Surname'],inplace=True)

In [79]:
# encode categorical gender
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [80]:
# Geography	Gender are two categorical colums so lets one hot encode it for no comparison 
# as it could happen in label encoder
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(df[['Geography']]).toarray()



In [81]:
geo_feature_names = onehot_encoder_geo.get_feature_names_out(['Geography'])
geo_feature_names

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [86]:
geo_encoded_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [87]:
#combine one hot encoder data with original data
df = pd.concat([df.drop('Geography',axis=1), geo_encoded_df], axis=1)

df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [69]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)
  

In [88]:
# divide the data in independent and dependent

x=df.drop('Exited',axis=1) # temporary
y=df['Exited']

# split 

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

# scale these features

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [72]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [73]:
!pip install tensorflow

In [21]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [22]:
# bulid your own awn model
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1] , )), # hidden lay1
    Dense(32,activation='relu'), # hidden lay1 
     Dense(1,activation='sigmoid')   
                                                
]

)

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
# COMPILE THE MODEL
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.01)
loss = 'binary_crossentropy'
metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [25]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
#Why? So you can run multiple experiments and
#  log each one in a separate folder for comparison.
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#Logs metrics like loss, accuracy, model graph, gradients, etc.
#  You can visualize them using: tensorboard --logdir=logs/fit

tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# SETUP EARLY STOPPING

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [26]:
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=100,
    callbacks=[tensorboard_callback, early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 6ms/step - loss: 396.7143 - accuracy: 0.6731 - val_loss: 124.7202 - val_accuracy: 0.4905
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 45.5511 - accuracy: 0.6812 - val_loss: 5.3561 - val_accuracy: 0.6960
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 2.0760 - accuracy: 0.7560 - val_loss: 0.5345 - val_accuracy: 0.8035
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5373 - accuracy: 0.7875 - val_loss: 0.4980 - val_accuracy: 0.8035
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5121 - accuracy: 0.7937 - val_loss: 0.5038 - val_accuracy: 0.8030
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5088 - accuracy: 0.7945 - val_loss: 0.4946 - val_accuracy: 0.8035
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5071 - accuracy: 0.7945 - val_loss: 0.4948 - val_accuracy:

In [27]:
model.save('model.h5')

c:\Users\KIIT0001\Desktop\ml_project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 9144), started 0:00:14 ago. (Use '!kill 9144' to kill it.)